In [5]:
from dotenv import load_dotenv
import openai
import os
import langchain.llms as llms
from langchain.chains import create_sql_query_chain
import pandas as pd
import sqlite3
from langchain_community.utilities import SQLDatabase
import openpyxl
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.prompts import FewShotPromptTemplate, PromptTemplate


In [2]:
working_dir = "C:/Users/nithi/OneDrive - University of Illinois Chicago/Documents/Capstone project/LLM project"
os.chdir(working_dir)

print(f"The current working directory: {os.getcwd()}")

The current working directory: C:\Users\nithi\OneDrive - University of Illinois Chicago\Documents\Capstone project\LLM project


In [4]:
# SQLite database connection
db = SQLDatabase.from_uri("sqlite:///dataset.db")

# GPT connection
llm = ChatOpenAI(model="gpt-4", temperature=0.2)

In [9]:
# Define the base prompt
base_prompt = PromptTemplate(
    input_variables=["input"],
    template="""You are a SQLite expert and Machine Learning Engineer. 
    Given an input question, create a syntactically correct SQLite query to run, to answer the input question.
    \n Only use the following tables: {table_info}.Question: {input}.Generate up to {top_k} SQL queries to answer the question.""",
)

# Define the metric prompt
metric_prompt = """**Metric Selection Instructions:**

* For questions about detailed performance metrics (accuracy, MAE, recall, precision), you MUST JOIN the 'models' table with the 'model_metrics_view' on the MODEL_ID column.

* Check the model type first:

   * For Regression models, use MAE from model_metrics_view.

   * For Classification models, use accuracy, precision, and recall from model_metrics_view.

    **Example:**  

    Question: Which models have recall greater than 0.9?

    SELECT Model_Name 
         FROM models 
         JOIN model_metrics_view ON models.MODEL_ID = model_metrics_view.MODEL_ID
         WHERE Model_type IN ('Multi-Class', 'Classification') and accuracy > 0.9;
  

    Question: What is the accuracy of Model 10?

     SELECT accuracy 
         FROM models 
         JOIN model_metrics_view ON models.MODEL_ID = model_metrics_view.MODEL_ID
         WHERE lower(Model_Name) = 'model 10' and Model_type IN ('Multi-Class', 'Classification');
    Answer: Model 10 is a Regression model type so it accuracy is not the right metric to calculate performace of the model
"""

# Define the data validation prompt
data_validation_prompt = """**Data Validation Instructions:**
* Before executing the query, ensure that the referenced data likely exists in the database.
* If potential issues are detected, indicate the problem and suggest alternatives instead of executing the query.
"""


# Prompt to answer the questions
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question in conversational tone.

      If the SQL result is empty, provide a helpful message indicating that no matching data was found.  

Question: {question}
SQL Query: {query}
SQL Result: {result}

Answer: """
)


In [10]:
# Combine the prompts using FewShotPromptTemplate
def generate_prompt(query):
    prompt = base_prompt

    # Analyze user query for keywords
    keywords = ["accuracy", "recall", "precision", "MAE", 'model', 'best', 'worst']
    detected_metrics = [metric for metric in keywords if metric in query.lower()]

    # Add conditional prompts based on detected metrics
    if detected_metrics:
        prompt = FewShotPromptTemplate(
            examples=[
                FewShotPromptTemplate.create_example(
                    base_prompt.template,
                    metric_prompt,
                ),
                FewShotPromptTemplate.create_example(
                    base_prompt.template,
                    data_validation_prompt,
                ),
            ],
            prefix=prompt.template,
            suffix="\nSQL Query:",
            example_prompt=base_prompt,
        )

    return prompt

prompt = generate_prompt("query")
print(prompt)


input_variables=['input', 'table_info', 'top_k'] template='You are a SQLite expert and Machine Learning Engineer. \n    Given an input question, create a syntactically correct SQLite query to run, to answer the input question.\n    \n Only use the following tables: {table_info}.Question: {input}.Generate up to {top_k} SQL queries to answer the question.'


In [11]:
# Initialize Chain 1: Generate SQL Query
generate_sql_chain = create_sql_query_chain(prompt=generate_prompt("dummy_query"), llm=llm, db=db)

execute_query = QuerySQLDataBaseTool(db=db)

# Initialize Chain 2: Execute SQL Query and generate structured answer
execute_sql_chain = answer_prompt | llm | StrOutputParser()

def calculate_token_size(text):
    # Split text into tokens and count the total number of tokens
    tokens = text.split()
    return len(tokens)

def execute_combined_chain(question):
    # Step 1: Generate SQL Query
    sql_query = generate_sql_chain.invoke({"question": question, "top_k": 1})

    # Step 2: Execute the SQL Query to get the result
    sql_result = execute_query(sql_query)  # Ensure this returns the result of executing the SQL query

    # Step 3: Pass the necessary inputs to the final chain and format the output to include both SQL query and result
    final_response = execute_sql_chain.invoke({"question": question, "query": sql_query, "result": sql_result})

    # Calculate token size of input and output
    input_token_size = calculate_token_size(question)
    output_token_size = calculate_token_size(final_response)
    total_token_size = input_token_size + output_token_size
    print(total_token_size)
    
    # Prompt user if total token size exceeds 10K tokens
    if total_token_size > 10000:
        return "Your query is too complex. Please try asking in a simpler way or split it into multiple questions."

    # Format the final answer to include both the SQL query and its result
    final_answer = f"SQL Query: {sql_query}\n Answer: {final_response}"
    return final_answer

In [13]:
# Example invocation
question = "Which model version is better in each model"
final_answer = execute_combined_chain(question)
print(final_answer)

323
SQL Query: SELECT Model_Name, Model_Version, MAX(Performance_Metrics) as Best_Performance
FROM models
GROUP BY Model_Name;
 Answer: The best version for each model based on their performance metrics are as follows: Model 1's best version is 1, Model 10's best version is 2, Model 11's best version is 1, Model 12's best version is 2, Model 13's best version is 3, Model 14's best version is 1, Model 15's best version is 1, Model 16's best version is 1, Model 17's best version is 2, Model 18's best version is 2, Model 19's best version is 1, Model 2's best version is 1, Model 20's best version is 2, Model 21's best version is 2, Model 22's best version is 1, Model 23's best version is 3, Model 24's best version is 3, Model 25's best version is 1, Model 26's best version is 2, Model 27's best version is 2, Model 28's best version is 3, Model 29's best version is 2, Model 3's best version is 3, Model 30's best version is 1, Model 31's best version is 3, Model 32's best version is 3, Mode